# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2340b60fd0>
├── 'a' --> tensor([[ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814]])
└── 'x' --> <FastTreeValue 0x7f2340b60f70>
    └── 'c' --> tensor([[ 0.2825, -1.5190,  0.0118,  0.1731],
                        [ 0.4305, -0.4464,  1.6371,  0.1959],
                        [ 0.9434,  1.6573, -0.0741,  1.9337]])

In [4]:
t.a

tensor([[ 0.8500,  0.6077, -1.0274],
        [ 1.7517, -0.3917,  0.0814]])

In [5]:
%timeit t.a

64.5 ns ± 0.716 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2340b60fd0>
├── 'a' --> tensor([[-0.1752, -1.9071, -1.0673],
│                   [-1.7437, -0.0749, -0.0083]])
└── 'x' --> <FastTreeValue 0x7f2340b60f70>
    └── 'c' --> tensor([[ 0.2825, -1.5190,  0.0118,  0.1731],
                        [ 0.4305, -0.4464,  1.6371,  0.1959],
                        [ 0.9434,  1.6573, -0.0741,  1.9337]])

In [7]:
%timeit t.a = new_value

69.8 ns ± 2.7 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8500,  0.6077, -1.0274],
               [ 1.7517, -0.3917,  0.0814]]),
    x: Batch(
           c: tensor([[ 0.2825, -1.5190,  0.0118,  0.1731],
                      [ 0.4305, -0.4464,  1.6371,  0.1959],
                      [ 0.9434,  1.6573, -0.0741,  1.9337]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8500,  0.6077, -1.0274],
        [ 1.7517, -0.3917,  0.0814]])

In [11]:
%timeit b.a

60.2 ns ± 0.819 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4118, -0.1574,  1.4726],
               [-0.6531,  1.5940, -0.8428]]),
    x: Batch(
           c: tensor([[ 0.2825, -1.5190,  0.0118,  0.1731],
                      [ 0.4305, -0.4464,  1.6371,  0.1959],
                      [ 0.9434,  1.6573, -0.0741,  1.9337]]),
       ),
)

In [13]:
%timeit b.a = new_value

549 ns ± 17.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

900 ns ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 246 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

267 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

232 µs ± 5.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f229580c460>
├── 'a' --> tensor([[[ 0.8500,  0.6077, -1.0274],
│                    [ 1.7517, -0.3917,  0.0814]],
│           
│                   [[ 0.8500,  0.6077, -1.0274],
│                    [ 1.7517, -0.3917,  0.0814]],
│           
│                   [[ 0.8500,  0.6077, -1.0274],
│                    [ 1.7517, -0.3917,  0.0814]],
│           
│                   [[ 0.8500,  0.6077, -1.0274],
│                    [ 1.7517, -0.3917,  0.0814]],
│           
│                   [[ 0.8500,  0.6077, -1.0274],
│                    [ 1.7517, -0.3917,  0.0814]],
│           
│                   [[ 0.8500,  0.6077, -1.0274],
│                    [ 1.7517, -0.3917,  0.0814]],
│           
│                   [[ 0.8500,  0.6077, -1.0274],
│                    [ 1.7517, -0.3917,  0.0814]],
│           
│                   [[ 0.8500,  0.6077, -1.0274],
│                    [ 1.7517, -0.3917,  0.0814]]])
└── 'x' --> <FastTreeValue 0x7f229580c700>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.2 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2340b59790>
├── 'a' --> tensor([[ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814],
│                   [ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814],
│                   [ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814],
│                   [ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814],
│                   [ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814],
│                   [ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814],
│                   [ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814],
│                   [ 0.8500,  0.6077, -1.0274],
│                   [ 1.7517, -0.3917,  0.0814]])
└── 'x' --> <FastTreeValue 0x7f229c522730>
    └── 'c' --> tensor([[ 0.2825, -1.5190,  0.0118,  0.1731],
                        [ 0.4305, -0.4464,  1.6371,  0.1959],
                 

In [23]:
%timeit t_cat(trees)

36.1 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

70.5 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2825, -1.5190,  0.0118,  0.1731],
                       [ 0.4305, -0.4464,  1.6371,  0.1959],
                       [ 0.9434,  1.6573, -0.0741,  1.9337]],
              
                      [[ 0.2825, -1.5190,  0.0118,  0.1731],
                       [ 0.4305, -0.4464,  1.6371,  0.1959],
                       [ 0.9434,  1.6573, -0.0741,  1.9337]],
              
                      [[ 0.2825, -1.5190,  0.0118,  0.1731],
                       [ 0.4305, -0.4464,  1.6371,  0.1959],
                       [ 0.9434,  1.6573, -0.0741,  1.9337]],
              
                      [[ 0.2825, -1.5190,  0.0118,  0.1731],
                       [ 0.4305, -0.4464,  1.6371,  0.1959],
                       [ 0.9434,  1.6573, -0.0741,  1.9337]],
              
                      [[ 0.2825, -1.5190,  0.0118,  0.1731],
                       [ 0.4305, -0.4464,  1.6371,  0.1959],
                       [ 0.9434,  1.6573, -0.0741,  1.9337]],

In [26]:
%timeit Batch.stack(batches)

94.8 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2825, -1.5190,  0.0118,  0.1731],
                      [ 0.4305, -0.4464,  1.6371,  0.1959],
                      [ 0.9434,  1.6573, -0.0741,  1.9337],
                      [ 0.2825, -1.5190,  0.0118,  0.1731],
                      [ 0.4305, -0.4464,  1.6371,  0.1959],
                      [ 0.9434,  1.6573, -0.0741,  1.9337],
                      [ 0.2825, -1.5190,  0.0118,  0.1731],
                      [ 0.4305, -0.4464,  1.6371,  0.1959],
                      [ 0.9434,  1.6573, -0.0741,  1.9337],
                      [ 0.2825, -1.5190,  0.0118,  0.1731],
                      [ 0.4305, -0.4464,  1.6371,  0.1959],
                      [ 0.9434,  1.6573, -0.0741,  1.9337],
                      [ 0.2825, -1.5190,  0.0118,  0.1731],
                      [ 0.4305, -0.4464,  1.6371,  0.1959],
                      [ 0.9434,  1.6573, -0.0741,  1.9337],
                      [ 0.2825, -1.5190,  0.0118,  0.1731],
                   

In [28]:
%timeit Batch.cat(batches)

170 µs ± 5.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

537 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
